# Pràctica 3 - PLH

### Realitzada pels alumnes Lluc Furriols i Pau Prat Moreno

In [2]:
'''
import os 
f = open("/dev/null", "w")
os.dup2(f.fileno(), 2)
f.close()

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
'''

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
import nltk
nltk.download('punkt', quiet=True) # Tokenitzador
nltk.download('averaged_perceptron_tagger', quiet=True) # Etiquetador POS
nltk.download('maxent_ne_chunker', quiet=True) # Etiquetador Entitats Anomenades
nltk.download('words', quiet=True)

True

In [ ]:
# Fet pel profe a la pissarra
'''
class FeatureGetter:
    def __init__(ws_cap, ws_prefix,...):
        self.ws_cap = ws_cap
        self.ws_prefix = ws_prefix
        ...
    def __call__(self, token, idx):
        pass
getter_1 = FeatureGetter(ws_cap=True, ws_prefix=False, ...)
'''

In [9]:
nltk.download('conll2002')
from nltk.corpus import conll2002

train_esp = conll2002.iob_sents('esp.train') # Train, 
val_esp = conll2002.iob_sents('esp.testa') # Val
test_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
val_ned = conll2002.iob_sents('ned.testa') # Val
test_ned = conll2002.iob_sents('ned.testb') # Test

# Convertir token postag classe --> postag classe (crec, ha dit 2)

[nltk_data] Downloading package conll2002 to /Users/pau/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!
